In [17]:
import pandas as pd
import os
from datetime import datetime

import matplotlib.pyplot as plt
import seaborn as sns
import os
import plotly.express as px
import pandas as pd

## Validation Information Table

In [ ]:
def Validation_info_Table() :
    val_df = pd.read_csv('./Case2_Validation/Validation_dataframe.csv')
    val_df = pd.DataFrame({'channel' : [], 'input_max' : [],
                        'input_min' : [], 'term_max' : [], 'before_time' : []})

    path = '../../Processing/data/Type_A/Pattern/'
    file_list = os.listdir(path)
    file_list_py = [file for file in file_list if file.endswith('.csv')] ## 파일명 끝이 .csv인 경우

    for i in file_list_py :
        df = pd.read_csv(path + i)
        try :
            time_list = []
            for i in range(len(df)) :
                past = datetime.strptime(df['삽입시간'][i], "%H:%M:%S")
                cur = datetime.strptime(df['삽입시간'][i+1], "%H:%M:%S")
                diff =  cur - past
                time_list.append(diff)
        except :
            '뒤에 날짜가 없습니다.'


        try :
            content = {'channel' : df['채널명'][0], 'input_max' : df['삽입시간'].max(),
                                'input_min' : df['삽입시간'].min(), 'term_max' : max(time_list).seconds, 'before_time' : None}
        except :
            30000                    
        val_df = val_df.append(content, ignore_index = True)
        val_df['term_max'] = val_df['term_max'].astype('int') 
        val_df = val_df.drop_duplicates()


        ### Visualization ###
        sns.set(font='NanumGothic',
        rc={"axes.unicode_minus":False, 'figure.figsize':(20,8)},
        style='darkgrid')

        target_df = pd.DataFrame(val_df["channel"].value_counts()).reset_index()
        target_df.sort_values(by ="channel" , ignore_index = True)

        TARGET = 'channel'

        target_df.columns = [TARGET, val_df['term_max']]

        ## keyword 개수가 30개 이상인 단어만 출력
        target_df = val_df
        fig = px.bar(data_frame =target_df, 
                    x = 'channel',
                    y = 'term_max' , 
                    color_continuous_scale="Emrld") 
        fig.show()
        target_df.sort_values(by ='term_max' , ignore_index = True)

    
    return val_df.to_csv('./Case2_Validation/Validation_dataframe.csv', index=False)

## Term_Max Visualization

In [23]:
### 예시 데이터 - 서울경제TV
data = pd.read_csv('../data/전체신호_cusotm.csv')
data1 = data[58:59]
data2 = data[61:62]

## Max_Term_Validation

In [24]:
## 이전 데이터 Validation 테이블에 저장 

def Save_Before_Data(data) :
    
    val_df = pd.read_csv('./Case2_Validation/Validation_dataframe.csv')
    for i in range(len(val_df)) :
        if val_df['channel'][i] == data['채널명'].iloc[0] :
            val_df['before_time'].iloc[i] = data['삽입시간'].iloc[0]
    return val_df.to_csv('./Case2_Validation/Validation_dataframe.csv', index=False)
    # val_df['input_max'] = val_df['input_max'].astype(int)  ## 수치 비교를 위한 타입 변경
    # val_df['input_min'] = val_df['input_min'].astype(int) 
    # data['삽입시간'] = data['삽입시간'].astype(int)
Save_Before_Data(data1)

/tmp/ipykernel_576014/1921963715.py:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [25]:
def Max_Term_Validation(data) :
    val_df = pd.read_csv('./Case2_Validation/Validation_dataframe.csv')

    diff = []
    for i in range(len(val_df)) :
        if val_df['channel'][i] == data['채널명'].iloc[0] :
            past = datetime.strptime(val_df['before_time'].iloc[i], "%H:%M:%S")
            current = datetime.strptime(data['삽입시간'].iloc[0], "%H:%M:%S")
            diff =  current - past
            diff = diff.seconds
    result = []
    for i in range(len(val_df)) :
        if val_df['channel'][i] == data['채널명'].iloc[0] :
            if val_df['term_max'].iloc[i] >= diff :
                answer = '정상데이터'
                result.append(answer)   
            elif val_df['term_max'].iloc[i] < diff :
                answer = '이상데이터'
                result.append(answer)  

    return result
Max_Term_Validation(data2)  



### return 값 설정 해야됨!!!!!!!!!!!!!!!

['정상데이터']

## Min Max Validation

In [26]:
def Min_Max_Validation(data) :
    
    val_df = pd.read_csv('./Case2_Validation/Validation_dataframe.csv')
    val = val_df

    ### 전처리 작업
    for i in range(len(val)) :                      ## 수치 비교를 위한 시간 전처리
        val['input_max'][i] = val['input_max'][i].replace(':', '') 
        val['input_min'][i] = val['input_min'][i].replace(':', '')
    for i in range(len(data)) :
        data['삽입시간'].iloc[i] = data['삽입시간'].iloc[i].replace(':', '')
    
    val['input_max'] = val['input_max'].astype(int)  ## 수치 비교를 위한 타입 변경
    val['input_min'] = val['input_min'].astype(int) 
    data['삽입시간'] = data['삽입시간'].astype(int)

    ### 모델링
    result = []
    for i in range(len(val)) :
        if val['channel'][i] == data['채널명'].iloc[0] :

            if val['input_max'][i] < data['삽입시간'].iloc[0] :
                print('이상 유무 : 이상데이터')
                answer = '이상데이터'
                result.append(answer)
            elif val['input_min'][i] > data['삽입시간'].iloc[0] :
                print('이상 유무 : 이상데이터')
                answer = '이상데이터'
                result.append(answer)  
            else :
                answer = '정상데이터'
                result.append(answer)               

    return result

In [27]:
Min_Max_Validation(data1)

/tmp/ipykernel_576014/1217689765.py:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_576014/1217689765.py:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_576014/1217689765.py:11: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_576014/1217689765.py:15: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value in

['정상데이터']